In [ ]:
import requests
import pandas as pd
import hvplot.pandas
import json
import io
import holoviews as hv
import geopandas as gp
import searvey
from dateutil.relativedelta import *

hv.extension('bokeh')

In [ ]:
# Define the base URL for the API
sl_api  = "https://webcritech.jrc.ec.europa.eu/SeaLevelsDb/api/Group/"
tad_api = "https://webcritech.jrc.ec.europa.eu/TAD_server/api/Groups/Get"

api_device = "https://webcritech.jrc.ec.europa.eu/api/Device/"

# Fetch the list of providers
sl_response = requests.get(sl_api)
tad_response = requests.get(tad_api)
sl_data = json.loads(sl_response.text)  # Parse the JSON response
tad_data = json.loads(tad_response.text)  # Parse the JSON response
sl_data;tad_data

SeaLevelDB stations

In [ ]:
# Initialize a list to hold all station data
SL_data = []

# Loop through the list of providers and fetch their stations
for provider in sl_data:
    provider_name = provider['Name']
    stations_url = sl_api + provider_name + "/Devices"
    stations_response = requests.get(stations_url)
    stations_data = json.loads(stations_response.text)  # Parse the JSON response
    # Loop through each station and extract data
    for device in stations_data:
        station_data = {
            'Provider': provider_name,
            'Id': device['Id'],
            'Name': device['Name'],
            'lat': device['Lat'],
            'lon': device['Lon'],
            'LastAccessStatus': device['CurrentStatus']['LastAccessStatus'],
            'LastDate': device['CurrentStatus']['LastDate'],
            'State': device['CurrentStatus']['State'],
            'SyncStatus': device['CurrentStatus']['SyncStatus'],
            'FileType': device['FileType'],
            'GroupId': device['GroupId'],
            'MovAvgNp': device['MovAvgNp'],
            'Notes': device.get('Notes'),  # Use .get() to handle missing keys
            'Source': device['Source']
            # Add other fields as necessary
        }
        SL_data.append(station_data)
SL_df = pd.DataFrame(SL_data)
SL_df
SL_df.to_csv('SeaLevelDb_stations.csv')
SL_df.groupby(['Provider','State']).count()[['Id']].hvplot.bar(
    rot = 90,
    logy = True,
    ylim = [0.5, 2000], 
    grid = True
).opts(
    width = 1200,
    height = 800,
    title = "SeaLevelDb data availability"
)

TAD_data Stations

In [ ]:
TAD_data = []
# Loop through the list of providers and fetch their stations
for provider in tad_data:
    provider_name = provider['Name']
    stations_url = f"https://webcritech.jrc.ec.europa.eu/TAD_server/api/Groups/GetGeoJSON?group={provider_name}&maxLatency=10000000"
    stations_response = requests.get(stations_url)
    stations_data = json.loads(stations_response.text)  # Parse the JSON response
    if isinstance(stations_data, dict): stations_data = [stations_data]
    # Loop through each station and extract data
    if len(stations_data)>0:
        for device in stations_data[0]['features']:
            properties = device.get('properties', {})
            geometry = device.get('geometry', {})
            coordinates = geometry.get('coordinates', [None, None])
            latency = properties.get('Latency', {}).get('Literal')
            last_data_date_str = properties.get('LastData', {}).get('Date')
            last_data_date = pd.Timestamp(last_data_date_str)
            
            # Determine if the measurement is within the last week
            state = "active" if last_data_date and (pd.Timestamp.now() - last_data_date) <= pd.Timedelta(days=7) else "inactive"

            station_data = {
                'Provider': properties.get('Provider'),
                'Id': device.get('id'),
                'Name': properties.get('Name'),
                'lat': coordinates[1],
                'lon': coordinates[0],
                'LastAccessStatus': properties.get('LastData', {}).get('Date'),
                'LastDate': last_data_date,
                'SyncStatus': latency,
                'State': state,
                'FileType': None,  # Update this if there is a corresponding field in the new data
                'GroupId': None,  # Update this if there is a corresponding field in the new data
                'MovAvgNp': None,  # Update this if there is a corresponding field in the new data
                'Notes': properties.get('Notes'),  # Use .get() to handle missing keys
                'Source': None
                # Add other fields as necessary
            }
            TAD_data.append(station_data)
TAD_df = pd.DataFrame(TAD_data)
TAD_df
TAD_df.to_csv('TAD_stations.csv')
TAD_df.groupby(['Provider','State']).count()[['Id']].hvplot.bar(
    rot = 90,
    logy = True,
    ylim = [0.5, 2000], 
    grid = True
).opts(
    width = 1600,
    height = 800,
    title = "SeaLevelDb data availability"
)

Reference stations already in use in Seareport: 

In [ ]:
def get_stofs2d_meta():
    stofs2d = pd.read_csv(
        "https://polar.ncep.noaa.gov/stofs/data/stofs_2d_glo_elev_stat_v2_1_0",
        names=["coords", "name"],
        sep="!",
        header=None,
        skiprows=1
    )
    stofs2d = stofs2d.assign(
        lon=stofs2d.coords.str.split("\t", n=1).str[0].astype(float),
        lat=stofs2d.coords.str.strip().str.rsplit("\t", n=1).str[1].astype(float),
        stofs2d_name=stofs2d.name.str.strip(),
    ).drop(columns=["coords", "name"])
    return stofs2d


def get_ioc_meta() -> gp.GeoDataFrame:
    meta_web = searvey.get_ioc_stations().drop(columns=["lon", "lat"])
    meta_api = (
        pd.read_json(
            "http://www.ioc-sealevelmonitoring.org/service.php?query=stationlist&showall=all"
        )
        .drop_duplicates()
        .drop(columns=["lon", "lat"])
        .rename(columns={"Code": "ioc_code", "Lon": "lon", "Lat": "lat"})
    )
    merged = pd.merge(
        meta_web,
        meta_api[["ioc_code", "lon", "lat"]].drop_duplicates(),
        on=["ioc_code"],
    )
    return merged

def merge_ioc_and_stofs(ioc: pd.DataFrame, stofs2d: pd.DataFrame) -> pd.DataFrame:
    stations = pd.concat((ioc, stofs2d), ignore_index=True)
    stations = stations.assign(unique_id=stations.ioc_code.combine_first(stations.stofs2d_name))
    return stations

ioc = get_ioc_meta()
stofs2d = get_stofs2d_meta()
stations = merge_ioc_and_stofs(ioc=ioc, stofs2d=stofs2d)
stations['is_sat'] = stations.unique_id.str.contains('SA')
stations['source'] = "stofs"
stations['source'][~stations.ioc_code.isna()] = "ioc"

Compare all stations

In [ ]:
plot0 = stations.drop(columns='geometry')[~stations['is_sat']][stations.source == 'stofs'].hvplot.points(
    x = 'lon',
    y='lat',
    c = 'orange',
    s=100,
    line_color = 'white',
    geo = True,
)
plot1 = stations.drop(columns='geometry')[~stations['is_sat']][stations.source == 'ioc'].hvplot.points(
    x = 'lon',
    y='lat',
    c = 'r',
    s=100,
    line_color = 'white',
    geo = True,
)
plot2 = pd.concat([SL_df,TAD_df]).hvplot.points(
    x = 'lon',
    y='lat',
    c = 'Provider',
    line_color = 'k',
    cmap = 'glasbey',
    geo = True,
    tiles="OSM",
)

(plot0 * plot1 * plot2).opts(
    width = 1400,
    height = 1070
)

Compare TD UNESCO with Seareport stations 

In [ ]:
plot0 = stations.drop(columns='geometry')[~stations['is_sat']][stations.source == 'stofs'].hvplot.points(
    x = 'lon',
    y='lat',
    c = 'orange',
    s=100,
    line_color = 'white',
    geo = True,
)
plot1 = stations.drop(columns='geometry')[~stations['is_sat']][stations.source == 'ioc'].hvplot.points(
    x = 'lon',
    y='lat',
    c = 'r',
    s=100,
    line_color = 'white',
    geo = True,
)
plot2 = SL_df[SL_df.Provider.isin(['TD UNESCO'])].hvplot.points(
    x = 'lon',
    y='lat',
    c = 'Provider',
    line_color = 'k',
    cmap = 'glasbey',
    geo = True,
    tiles="OSM",
)

(plot0 * plot1 * plot2).opts(
    width = 1400,
    height = 1070
)

In [ ]:
all_stations = pd.concat([SL_df,TAD_df])
all_stations['period_inactive'] = all_stations.apply(lambda x:(pd.Timestamp.now() - pd.Timestamp(x['LastDate']).tz_localize(None)).total_seconds()/3600/24/365.25, axis = 1)

# Updated code using .loc to avoid SettingWithCopyWarning
all_stations.loc[all_stations.period_inactive <= 0, 'period_inactive'] = 0
all_stations.loc[all_stations.period_inactive >= 10, 'period_inactive'] = 10

# Then you can describe the data
all_stations.period_inactive.hvplot.hist().opts(title = "years inactive")

In [ ]:
all_stations.to_csv('TAD_SLDB_stations.csv')
all_stations.hvplot.points(
    x = 'lon',
    y='lat',
    c = 'period_inactive',
    line_color = "k",
    line_width = 0.25,
    cmap = 'fire_r',
    S = 100,
    geo = True,
    tiles="OSM",
).opts(title = "years inactive", width = 1400, height=900)

In [ ]:
all_stations['LastDate_pd'] =  all_stations.apply(lambda x:pd.Timestamp(x['LastDate']).tz_localize(None), axis = 1)
all_stations['LastDate_pd'].hvplot.hist(bins = 100)

Lots of station seem to have stopped in January 2024

### get data (not finished yet)

In [ ]:
from searvey.multi import multithread

# rate_limit 
def fetch_SeaLevelDb_station(tmin: pd.Timestamp, tmax: pd.Timestamp, id: int, NSTEPS = 10000000):
    url = f"https://webcritech.jrc.ec.europa.eu/SeaLevelsDb/api/Device/{id}/Data?tMin={tmin.strftime('%Y-%m-%d')}&tMax={tmax.strftime('%Y-%m-%d')}&nPts={NSTEPS}&field=level&mode=CSV"
    stations_response = requests.get(url)
    if stations_response.ok:
        data = json.loads(stations_response.text)  # Use `loads` instead of `load`
        df = pd.DataFrame(data)
        return df
    else:
        print(f"Error: {stations_response.status_code}")
        return pd.DataFrame()


def fetch_TAD_station(tmin: pd.Timestamp, tmax: pd.Timestamp, id: int, NSTEPS = 10000000): 
    url = f"https://webcritech.jrc.ec.europa.eu/TAD_server/api/Data/Get/{id}?tMin={tmin.strftime('%Y-%m-%d')}&tMax={tmax.strftime('%Y-%m-%d')}&nRec={NSTEPS}&mode=CSV"
    stations_response = requests.get(url)
    if stations_response.ok:
        data = io.StringIO(stations_response.text)  # Create a text stream object
        df = pd.read_csv(data)  # Read the DataFrame from the text stream
        return df
    else:
        print(f"Error: {stations_response.status_code}")
        return pd.DataFrame()


def get_TAD_stations(df:pd.DataFrame, device_var: str):
    for s, station in df.iterrows():
        print(s, station)
        1/0


def get_SLDB_stations(df:pd.DataFrame, device_var: str):
    for s, station in df.iloc[::-1].iterrows():
        print(station)
        func_kwargs = []
        for tstep in pd.date_range(pd.Timestamp(2020, 1, 1), station.LastDate, freq='MS'):
            tmin = tstep
            tmax = tstep + relativedelta(months=+1)
            func_kwargs.append(
                dict(
                    tmin=tmin,
                    tmax=tmax,
                    Device=station[device_var],
                ),
            )
        results = multithread(
            func=fetch_SeaLevelDb_station,
            func_kwargs=func_kwargs,
            n_workers=10,
            print_exceptions=False,
            disable_progress_bar=False,
        )   
        for result in results:
            if result.result is not None:
                df = result.result
                print(df)
        1/0

get_SLDB_stations(SL_df, "Id")

In [27]:
Device = 94
url = f"https://webcritech.jrc.ec.europa.eu/TAD_server/api/Data/Get/{Device}?tMin=2024-08-01&tMax=2024-09-01&nRec={10000000}&mode=CSV"

In [ ]:
stations_response = requests.get(url)
data = io.StringIO(stations_response.text)  # Create a text stream object
df = pd.read_csv(data)  # Read the DataFrame from the text stream
df

In [ ]:
df.hvplot.line( y='Lev RAD (m)')